<a href="https://colab.research.google.com/github/pvt-16/kaggle-ariel-data-challenge/blob/master/ariel_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd


c:\Python311\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [26]:
input_path = 'Dataset/Metadata/'

### Metadata files

In [27]:
labels_df = pd.read_csv(f'{input_path}train_labels.csv')
wavelengths = pd.read_csv(f'{input_path}wavelengths.csv')
test_adc_info = pd.read_csv(f'{input_path}test_adc_info.csv')
train_adc_info = pd.read_csv(f'{input_path}train_adc_info.csv')
axis_info = pd.read_parquet(f'{input_path}axis_info.parquet')


In [48]:
PLANET_ID = 'planet_id'

In [47]:
sel_planet = '1011759019'

In [29]:
print(labels_df.shape, wavelengths.shape, test_adc_info.shape, train_adc_info.shape, axis_info.shape)

(673, 284) (1, 283) (1, 6) (673, 6) (135000, 4)


In [49]:
test_adc_info

,planet_id,FGS1_adc_offset,FGS1_adc_gain,AIRS-CH0_adc_offset,AIRS-CH0_adc_gain,star
0,499191466,-331.033009,0.823442,-537.6921,0.937944,1


In [53]:
train_adc_info

,planet_id,FGS1_adc_offset,FGS1_adc_gain,AIRS-CH0_adc_offset,AIRS-CH0_adc_gain,star
0,785834,-343.335938,0.837244,-778.916533,0.924746,1
1,14485303,-366.381992,0.842983,-740.323212,0.931727,1
2,17002355,-386.107037,1.041701,-808.690659,1.513541,0
3,24135240,-339.737490,0.840239,-776.124167,0.931277,1
4,25070640,-355.660433,1.041280,-848.371494,1.503923,0
...,...,...,...,...,...,...
668,4249337798,-343.249428,0.856713,-881.907525,0.931115,1
669,4257395405,-321.834060,1.043631,-824.457988,1.523148,0
670,4266129805,-359.583003,1.027149,-883.219918,1.517923,0
671,4273166473,-340.156841,1.037533,-856.451284,1.519154,0


In [52]:
train_adc_info[train_adc_info[PLANET_ID] == sel_planet]

,planet_id,FGS1_adc_offset,FGS1_adc_gain,AIRS-CH0_adc_offset,AIRS-CH0_adc_gain,star


In [42]:
axis_info.sample(20)

,AIRS-CH0-axis0-h,AIRS-CH0-axis2-um,AIRS-CH0-integration_time,FGS1-axis0-h
119212,NaN,NaN,NaN,6.622917
66848,NaN,NaN,NaN,3.713806
22179,NaN,NaN,NaN,1.232167
74582,NaN,NaN,NaN,4.143472
114998,NaN,NaN,NaN,6.388806
76905,NaN,NaN,NaN,4.272500
63470,NaN,NaN,NaN,3.526139
53954,NaN,NaN,NaN,2.997472
67697,NaN,NaN,NaN,3.760944
57756,NaN,NaN,NaN,3.208694


In [40]:
wavelengths

,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
0,0.705,1.951761,1.960612,1.96945,1.978273,1.987083,1.995879,2.004661,2.01343,2.022184,...,3.84972,3.85481,3.859887,3.86495,3.869999,3.875034,3.880055,3.885063,3.890056,3.895036


In [25]:
labels_df[0:1]

,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
0,785834,0.001086,0.001137,0.001131,0.001124,0.001138,0.001131,0.001123,0.001127,0.00112,...,0.001075,0.001076,0.001076,0.001076,0.001074,0.001073,0.001072,0.001073,0.001073,0.001072


In [ ]:
labels_df.sort_values(PLANET_ID)

,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283
0,785834,0.001086,0.001137,0.001131,0.001124,0.001138,0.001131,0.001123,0.001127,0.001120,...,0.001075,0.001076,0.001076,0.001076,0.001074,0.001073,0.001072,0.001073,0.001073,0.001072
1,14485303,0.001835,0.001835,0.001834,0.001833,0.001833,0.001833,0.001833,0.001834,0.001834,...,0.001893,0.001892,0.001892,0.001891,0.001891,0.001891,0.001890,0.001890,0.001889,0.001888
2,17002355,0.002792,0.002814,0.002808,0.002804,0.002809,0.002805,0.002802,0.002805,0.002801,...,0.002784,0.002783,0.002783,0.002783,0.002783,0.002784,0.002784,0.002785,0.002785,0.002784
3,24135240,0.001294,0.001308,0.001308,0.001306,0.001306,0.001303,0.001306,0.001314,0.001314,...,0.001405,0.001404,0.001403,0.001402,0.001401,0.001400,0.001399,0.001397,0.001395,0.001393
4,25070640,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,0.001987,...,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988,0.001988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,4249337798,0.000790,0.000828,0.000825,0.000822,0.000830,0.000826,0.000822,0.000823,0.000820,...,0.000798,0.000799,0.000799,0.000799,0.000798,0.000796,0.000796,0.000796,0.000796,0.000796
669,4257395405,0.004063,0.004094,0.004090,0.004094,0.004083,0.004069,0.004076,0.004097,0.004100,...,0.004042,0.004042,0.004041,0.004041,0.004041,0.004041,0.004041,0.004040,0.004040,0.004039
670,4266129805,0.006566,0.006569,0.006568,0.006568,0.006567,0.006567,0.006568,0.006570,0.006569,...,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566,0.006566
671,4273166473,0.005647,0.005700,0.005687,0.005676,0.005687,0.005676,0.005667,0.005673,0.005663,...,0.005603,0.005603,0.005603,0.005602,0.005602,0.005604,0.005606,0.005607,0.005607,0.005605


In [ ]:

labels_df[labels_df[PLANET_ID]== sel_planet]

,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283


In [ ]:
[train/test]/[planet_id]/[AIRS-CH0/FGS1]_calibration/flat.parquet:

,planet_id,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,...,wl_274,wl_275,wl_276,wl_277,wl_278,wl_279,wl_280,wl_281,wl_282,wl_283


In [ ]:
pd.read_parquet(f'train/{sel_planet}/AIRS-CH0_signal.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'train/1011759019/AIRS-CH0_signal.parquet'